## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
0,0,Paita,-5.0892,-81.1144,82.40,61,0,11.50,clear sky,PE,2021-04-10 21:29:56
1,1,Tiarei,-17.5333,-149.3333,87.80,62,75,5.75,light rain,PF,2021-04-10 21:29:57
2,2,Dingle,10.9995,122.6711,74.73,87,68,6.02,light rain,PH,2021-04-10 21:29:57
3,3,Punta Arenas,-53.1500,-70.9167,53.60,58,40,6.91,scattered clouds,CL,2021-04-10 21:26:00
4,4,Rikitea,-23.1203,-134.9692,78.73,72,0,10.02,light rain,PF,2021-04-10 21:29:57


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
2,2,Dingle,10.9995,122.6711,74.73,87,68,6.02,light rain,PH,2021-04-10 21:29:57
3,3,Punta Arenas,-53.1500,-70.9167,53.60,58,40,6.91,scattered clouds,CL,2021-04-10 21:26:00
4,4,Rikitea,-23.1203,-134.9692,78.73,72,0,10.02,light rain,PF,2021-04-10 21:29:57
5,5,Durant,33.9940,-96.3708,71.60,33,1,17.27,clear sky,US,2021-04-10 21:29:58
6,6,Mataura,-46.1927,168.8643,60.01,91,65,3.00,broken clouds,NZ,2021-04-10 21:29:58
8,8,Hermanus,-34.4187,19.2345,61.00,91,100,1.01,overcast clouds,ZA,2021-04-10 21:29:59
9,9,Hilo,19.7297,-155.0900,77.00,57,75,8.05,broken clouds,US,2021-04-10 21:29:59
10,10,Kapaa,22.0752,-159.3190,77.00,57,75,10.36,broken clouds,US,2021-04-10 21:29:59
14,14,Vila Franca Do Campo,37.7167,-25.4333,62.60,94,75,9.22,mist,PT,2021-04-10 21:30:01
15,15,Esperance,-33.8667,121.9000,57.20,88,98,11.50,overcast clouds,AU,2021-04-10 21:30:02


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Country                2
Date                   0
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [36]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Country                0
Date                   0
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Dingle,PH,74.73,light rain,10.9995,122.6711,
3,Punta Arenas,CL,53.60,scattered clouds,-53.1500,-70.9167,
4,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,
5,Durant,US,71.60,clear sky,33.9940,-96.3708,
6,Mataura,NZ,60.01,broken clouds,-46.1927,168.8643,
8,Hermanus,ZA,61.00,overcast clouds,-34.4187,19.2345,
9,Hilo,US,77.00,broken clouds,19.7297,-155.0900,
10,Kapaa,US,77.00,broken clouds,22.0752,-159.3190,
14,Vila Franca Do Campo,PT,62.60,mist,37.7167,-25.4333,
15,Esperance,AU,57.20,overcast clouds,-33.8667,121.9000,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except:
        print("Hotel not found... skipping.")    

print("------- All Done!")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
------- All Done!


In [65]:
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Dingle,PH,74.73,light rain,10.9995,122.6711,SEGAYA PROPERTY
3,Punta Arenas,CL,53.60,scattered clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho
4,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,Pension Maro'i
5,Durant,US,71.60,clear sky,33.9940,-96.3708,Holiday Inn Express & Suites Durant
6,Mataura,NZ,60.01,broken clouds,-46.1927,168.8643,Ellie's Villa
...,...,...,...,...,...,...,...
678,Doha,QA,77.00,clear sky,25.2867,51.5333,Movenpick Hotel Doha
679,Beyneu,KZ,60.60,scattered clouds,45.3167,55.2000,NUR
681,Quzhou,CN,55.96,light rain,28.9594,118.8686,Quzhou Jiujiu Business Hotel
684,Alangayam,IN,69.66,clear sky,12.6000,78.7500,Sri Bhuvaneshwari Mahal


In [72]:
# 7. Drop the rows where there is no Hotel Name.
is_nan = hotel_df.isnull().sum()
is_nan

City                   0
Country                0
Max Temp               0
Weather Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [76]:
new_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [78]:
new_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Dingle,PH,74.73,light rain,10.9995,122.6711,SEGAYA PROPERTY
3,Punta Arenas,CL,53.60,scattered clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho
4,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,Pension Maro'i
5,Durant,US,71.60,clear sky,33.9940,-96.3708,Holiday Inn Express & Suites Durant
6,Mataura,NZ,60.01,broken clouds,-46.1927,168.8643,Ellie's Villa
...,...,...,...,...,...,...,...
678,Doha,QA,77.00,clear sky,25.2867,51.5333,Movenpick Hotel Doha
679,Beyneu,KZ,60.60,scattered clouds,45.3167,55.2000,NUR
681,Quzhou,CN,55.96,light rain,28.9594,118.8686,Quzhou Jiujiu Business Hotel
684,Alangayam,IN,69.66,clear sky,12.6000,78.7500,Sri Bhuvaneshwari Mahal


In [79]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
new_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [80]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = new_hotel_df[["Lat", "Lng"]]

In [81]:
# 11a. Add a marker layer for each city to the map. 
max_temp = new_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))